In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

# Load orders CSV
orders = pd.read_csv("orders.csv")

print("Orders Data")
print(orders.head())


Orders Data
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [2]:
# Load users JSON
users = pd.read_json("users.json")

print("Users Data")
print(users.head())


Users Data
   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [3]:
import sqlite3

# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Read SQL file
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

# Execute SQL
cursor.executescript(sql_script)

# Load restaurant table into pandas
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print("Restaurants Data")
print(restaurants.head())


Restaurants Data
   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [4]:
# Merge orders with users
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

# Merge result with restaurants
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [5]:
# Save final dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

print("Final Dataset Created Successfully!")
print(final_df.head())


Final Dataset Created Successfully!
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.

In [6]:
final_df.groupby("order_date")["order_id"].count()


order_date
01-01-2023    29
01-01-2024    27
01-02-2023    25
01-03-2023    37
01-04-2023    32
              ..
31-05-2023    33
31-07-2023    27
31-08-2023    24
31-10-2023    30
31-12-2023    29
Name: order_id, Length: 366, dtype: int64

In [9]:
print(final_df.columns)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')


In [10]:
final_df.groupby(["city", "cuisine"])["total_amount"].sum()


city       cuisine
Bangalore  Chinese    525046.13
           Indian     543014.04
           Italian    567881.80
           Mexican    571004.61
Chennai    Chinese    491710.42
           Indian     441946.13
           Italian    522051.88
           Mexican    534804.60
Hyderabad  Chinese    470061.40
           Indian     467210.85
           Italian    452008.82
           Mexican    500085.51
Pune       Chinese    443686.70
           Indian     519241.56
           Italian    482261.30
           Mexican    479608.37
Name: total_amount, dtype: float64

In [12]:
gold_revenue_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(gold_revenue_city)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64


In [13]:
avg_order_cuisine = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(avg_order_cuisine)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64


In [14]:
high_value_users = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)

count_users = (high_value_users > 1000).sum()
print(count_users)


2544


In [15]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels)

rating_revenue = (
    final_df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(rating_revenue)


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64


In [17]:
gold_avg_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(gold_avg_city)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64


In [19]:
restaurant_count = final_df.groupby("cuisine")["restaurant_id"].nunique()
revenue = final_df.groupby("cuisine")["total_amount"].sum()

summary = pd.DataFrame({
    "restaurants": restaurant_count,
    "revenue": revenue
}).sort_values(by=["restaurants", "revenue"], ascending=[True, False])

print(summary)


         restaurants     revenue
cuisine                         
Chinese          120  1930504.65
Italian          126  2024203.80
Indian           126  1971412.58
Mexican          128  2085503.09


In [20]:
gold_orders = final_df[final_df["membership"] == "Gold"].shape[0]
total_orders = final_df.shape[0]

percentage = round((gold_orders / total_orders) * 100)
print(percentage)


50


In [22]:
restaurant_stats = (
    final_df
    .groupby("restaurant_name_y")
    .agg(
        avg_order=("total_amount", "mean"),
        total_orders=("order_id", "count")
    )
)

filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]
print(filtered.sort_values("avg_order", ascending=False))


                     avg_order  total_orders
restaurant_name_y                           
Restaurant_294     1040.222308            13
Restaurant_262     1029.473333            18
Restaurant_77      1029.180833            12
Restaurant_193     1026.306667            15
Restaurant_7       1002.140625            16
...                        ...           ...
Restaurant_184      621.828947            19
Restaurant_498      596.815556            18
Restaurant_192      589.972857            14
Restaurant_329      578.578667            15
Restaurant_300      572.686471            17

[241 rows x 2 columns]


In [23]:
final_df[final_df["restaurant_name_y"] == "Restaurant_294"][
    ["restaurant_name_y", "cuisine", "rating"]
].drop_duplicates()


,restaurant_name_y,cuisine,rating
1407,Restaurant_294,Italian,3.1


In [24]:
final_df.groupby(["membership", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [25]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.quarter

final_df.groupby("quarter")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


C:\Users\MANOHAR SINGAM\AppData\Local\Temp\ipykernel_15236\397454376.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [26]:
gold_orders_count = final_df[final_df["membership"] == "Gold"].shape[0]
gold_orders_count


4987

In [27]:
hyderabad_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)


1889367

In [28]:
distinct_users = final_df["user_id"].nunique()
distinct_users


2883

In [29]:
gold_avg_order_value = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
round(gold_avg_order_value, 2)


797.15

In [30]:
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [31]:
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city


'Chennai'

In [32]:
orders_top_gold_city = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]

orders_top_gold_city


1337